# Detecting the difficulty level of French texts




In [1]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [2]:
! pip install kaggle

In [3]:
!mkdir ~/.kaggle

In [4]:
!cp /content/drive/MyDrive/kaggle.json ~/.kaggle/kaggle.json

In [5]:
! kaggle competitions download -c detecting-french-texts-difficulty-level-2023
from zipfile import ZipFile
with ZipFile('detecting-french-texts-difficulty-level-2023.zip','r') as zip:
  zip.extractall(path="")

  0% 0.00/303k [00:00<?, ?B/s]
100% 303k/303k [00:00<00:00, 79.8MB/s]


In [6]:
!pip install -U spacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 17.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.1/50.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.0/45.0 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: spacy
    Found existing installation: spacy 3.6.1
    Uninstalling spacy-3.6.1:
      Successfully uninstalled spacy-3.6.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
en-core-web-sm 3.6.0 requires spacy<3.7.0,>=3.6.0, but you have spacy 3.7.2 which is incompatible.


In [7]:
!python -m spacy download fr_core_news_sm

2023-11-20 15:23:06.094629: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-20 15:23:06.094706: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-20 15:23:06.094760: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-20 15:23:06.109478: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-11-20 15:23:10.277544: W tensorflow/compiler/

In [8]:
import pandas as pd
import numpy as np

In [9]:
train_data = pd.read_csv('training_data.csv', index_col = 'id')

In [10]:
train_data.head()

,sentence,difficulty
id,,
0,Les coûts kilométriques réels peuvent diverger...,C1
1,"Le bleu, c'est ma couleur préférée mais je n'a...",A1
2,Le test de niveau en français est sur le site ...,A1
3,Est-ce que ton mari est aussi de Boston?,A1
4,"Dans les écoles de commerce, dans les couloirs...",B1


In [11]:
train_data.isnull().sum()

sentence      0
difficulty    0
dtype: int64

In [12]:
train_data.duplicated().sum()

0

In [13]:
X_data = train_data['sentence']

In [14]:
y_data = train_data['difficulty']

In [15]:
from sklearn.model_selection import train_test_split

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X_data, y_data, test_size=0.2, random_state=42)

In [17]:
y_train.shape[0]

3840

In [18]:
X_train.shape[0]

3840

In [19]:
import spacy
from spacy import displacy
import string
from nltk.stem.snowball import SnowballStemmer

In [20]:
#french language
french = spacy.load('fr_core_news_sm')
stemmer = SnowballStemmer(language='french')

In [21]:
test = "Bouygues a eu 1 coupure de réseau à Marseille."
test2 = "Par ailleurs, 42 % des adolescents déclarent qu'ils seraient dévastés s'ils devaient "

In [22]:
def spacy_tokenizer(sentence):

    punctuations = string.punctuation
    stop_words = spacy.lang.fr.stop_words.STOP_WORDS

    # Create token object, which is used to create documents with linguistic annotations.
    mytokens = french(sentence)

    # Remove proper nouns (noms propres)
    mytokens = [word for word in mytokens if word.pos_ != "PROPN"]

    # Lemmatize each token and convert each token into lowercase
    mytokens = [stemmer.stem(X.text) for X in mytokens]

    # Remove numbers
    mytokens = [word for word in mytokens if not word.isdigit()]

    # Remove stop words and punctuation
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]

    # Remove anonymous dates and people
    #mytokens = [ word.replace('xx/', '').replace('xxxx/', '').replace('xx', '') for word in mytokens ]
    #mytokens = [ word for word in mytokens if word not in ["xxxx", "xx", ""] ]
    # Return preprocessed list of tokens
    return mytokens

In [23]:
import re

In [24]:
def spacy_tokenizerV2(sentence):

    punctuations = string.punctuation
    stop_words = spacy.lang.fr.stop_words.STOP_WORDS
    BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
    # Tokenize the sentence and remove stop words
    #tokens = [word for word in sentence.split() if word.lower() not in stop_words]

    mytokens = french(sentence)
    mytokens = [ word.lemma_.lower().strip() if word.lemma_ != "-PRON-" else word.lower_ for word in mytokens ]
    mytokens = [ word for word in mytokens if word not in stop_words and word not in punctuations ]
    # Join the tokens into a single string
    #processed_sentence = ' '.join(mytokens)

    # Return preprocessed list of tokens
    return mytokens

In [25]:
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(spacy.lang.fr.stop_words.STOP_WORDS)

def clean_text(text):
    """
        text: a string

        return: modified initial string
    """
    text = text.lower() # lowercase text
    #text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    #text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text

In [26]:
spacy_tokenizerV2(test2)

['ailleurs', '42', 'pourcent', 'adolescent', 'déclarer', 'dévaster', 'devoir']

In [27]:
clean_text(test2)

"ailleurs, 42 % adolescents déclarent qu'ils dévastés s'ils devaient"

In [28]:
X_train

id
2758             On n'en sort pas, et moi, j'en ai marre.
2850             Il y avait deux enfants de 14 et 11 ans.
4440            Son appartement est loin de l'Université.
4576    Les négociateurs du Congrès de Vienne établire...
1774           Je ne trouve pas le livre que tu cherches.
                              ...                        
4426                       C'est très gentil de sa part !
466     Alexandre aimait réviser à l'université pour f...
3092    Socrate incarne, en effet, le sage qui sait di...
3772               Nous sommes en vacances chez des amis.
860      Tu était déjà allé voir cette pièce de théâtre ?
Name: sentence, Length: 3840, dtype: object

In [29]:
X_train = X_train.apply(clean_text)

In [30]:
test2.split()

['Par',
 'ailleurs,',
 '42',
 '%',
 'des',
 'adolescents',
 'déclarent',
 "qu'ils",
 'seraient',
 'dévastés',
 "s'ils",
 'devaient']

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier

In [32]:

from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfTransformer, CountVectorizer
from sklearn.preprocessing import StandardScaler

## Définissez le vecteur TF-IDF avec votre fonction de tokenization
#tfidf = TfidfVectorizer( min_df=5, max_df=0.90, norm='l2', encoding='latin-1', ngram_range=(1, 2), tokenizer=spacy_tokenizerV2)
#
## Ajoutez la normalisation avec StandardScaler
#classifier = LogisticRegression(solver='lbfgs', max_iter=500)
#
# #Créez le pipeline avec la normalisation
#pipe = Pipeline([
#    ('vectorizer', tfidf),
#    ('classifier', classifier)
#])
pipe = Pipeline([
    ('vect', CountVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', LogisticRegression(max_iter=1000)),
])

# Ajustez le modèle sur l'ensemble d'entraînement
pipe.fit(X_train, y_train)


Pipeline(steps=[('vect', CountVectorizer()), ('tfidf', TfidfTransformer()),
                ('clf', LogisticRegression(max_iter=1000))])

In [33]:
bow = CountVectorizer(stop_words=stop_words_french).fit_transform(X_train[:3])

# Show feature matrix
print(bow.toarray())

NameError: ignored

In [ ]:
bow = CountVectorizer(tokenizer=spacy_tokenizerV2).fit_transform(X_train[:3])

# Show feature matrix
print(bow.toarray())

In [ ]:
y_pred = pipe.predict(X_test)

In [ ]:
y_pred

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score, f1_score, precision_score, recall_score

In [ ]:
print(f'The accuracy using TF-IDF and Logistic Regression is: {round(accuracy_score(y_test, y_pred), 3)}')
#print(f'Accuracy of the best model: {accuracy_tfidf * 100:.2f}%')
print(f'\nThe precision using TF-IDF and Logistic Regression is: {round(precision_score(y_test, y_pred,average="macro"), 3)}')
print(f'\nThe recall using TF-IDF and Logistic Regression is: {round(recall_score(y_test, y_pred,average="macro"), 3)}')
print(f'\nThe F1-score using TF-IDF and Logistic Regression is: {round(f1_score(y_test, y_pred,average="macro"), 3)}')


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
## Confusion matrix
conf_mat = confusion_matrix(y_test, y_pred)
fig, ax = plt.subplots(figsize=(8,7))
sns.heatmap(conf_mat, annot=True, fmt='d')
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()